In [1]:
import itertools
import sys
import json
import warnings
warnings.filterwarnings("ignore")

from util import *
from scipy.sparse import vstack, csr_matrix, coo_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from sklearn.utils import class_weight
from keras.layers import *
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, CSVLogger

Using TensorFlow backend.


# Corpora for training model

In [2]:
sources = {'art': [60114472, 19191317], 
           'politics': [29534144, 23482909], 
           'finances': [62438886, 81354264],
           'strateg_management': ["http://www.stplan.ru/", "http://www.30n.ru/2/1.html"], 
           'law': [65974416, 79084019],
           'elaboration': ["https://vk.com/"], 
           'industry': [67183197, 66233468], 
           'education': [30558759, 98643656],
#            'social_business': [23509868, 56821139],
           'charity': [],
           'public_health': [78860407, 61490488],
           'agriculture': ["http://www.nsh.ru/", "http://россельхоз.рф/"],
           'government_management': ["http://be5.biz/upravlenie/gosudarstvennoe_upravlenie.html", 97296142],
           'smm': [74686342, 79925455],
           'innovations': [98643656, 63337812],
           'safety': [37959220, 10933209],
           'military': ["http://www.soldiering.ru", "https://voennoe-delo.com"],
           'corporative_management': ["http://www.cfin.ru/management/"],
           'social_safety': [49582956, 72388807],
           'building': [30713157, 26978036],
           'entrepreneurship': [69560028, 73537456],
           'sport': [29809500, 128350290],
           'investitions': [37876217, 3800580]
          }

In [3]:
# getting data for corpora
for tag, ids_ in sources.items():
    path = f"assets/corpora/{tag}.txt"
    s = set()
    if not os.path.exists(path):
        with open(path, "w") as f:
            for id_ in ids_:
                if isinstance(id_, int):
                    wall = ParseClass.getallwall({"owner_id": -id_}, 1000)
                    for post in tqdm.tqdm(wall):
                        if len(post) and post not in s:
                            s.add(post)
                            _ = f.write(f"{post}\n")
                elif isinstance(id_, str):
                    links = np.random.choice(
                        list(ParseClass.get_all_links(id_)), 1000)
                    for link in tqdm.tqdm(links):
                        try:
                            page = requests.get(link).text
                            soup = BeautifulSoup(page, "lxml")
                            for text in soup.text.strip().split("\n"):
                                if len(text) and text not in s:
                                    s.add(text)
                                    _ = f.write(f"{text}\n")
                        except:
                            pass

Manual processing

In [4]:
categories = list(sources.keys())

In [5]:
labels = ["Искусство", "Политика", "Финансы", "Стратегическое управление", "Юриспруденция", "Исследования и разработки",
          "Промышленность", "Образование", "Благотворительность", "Здравоохранение", "Сельское хозяйство", 
          "Государственное управление", "Реклама и маркетинг", "Инновации и модернизация", "Безопасность", 
          "Военное дело", "Корпоративное управление", "Социальная защита", "Строительство", "Предпринимательство",
          "Спорт", "Инвестиции"]
norm_names = dict(zip(categories, labels))
norm_names_reversed = dict([(a[1], a[0]) for a in norm_names.items()])

In [6]:
# df = pd.read_msgpack("assets/df.msg")
# competencies = pd.read_csv("assets/competencies.csv")
# competencies = competencies.dropna()[['Id', 'Интересы']]
# social = pd.read_excel("assets/social.xlsx", names=['id', 'name', 'fb', 'ok', 'vk', 'email', 'twitter'])
# accepted = social.fb.dropna().index | social.vk.dropna().index
# social = social.loc[accepted, ['name', 'id', 'vk', 'fb']]
# social.vk = social.vk.apply(lambda x: str(x).split("/")[-1])
# social.fb = social.fb.apply(lambda x: str(x).split("/")[-1])
# social.fb = social.fb.apply(lambda x: str(x).split("id=")[-1])
# social.fb = social.fb.apply(lambda x: str(x).split("?ref=")[0])
# social.fb = social.fb.replace('nan', np.nan)
# social = social.merge(df[['vk', 'vk_id']], how='outer', on='vk')
# social.vk = social.vk_id.fillna(0)
# social = social.replace(0, np.nan).drop(labels=['vk_id'], axis=1)
# social = social.set_index('id').merge(how='inner', right=competencies.set_index('Id'), left_index=True, right_index=True)
# to_exclude = pd.read_csv("assets/known_users.csv")['Leader-ID'].dropna().astype('int').values
# social = social.loc[~social.index.isin(to_exclude), :]
# social.to_msgpack("assets/social.msg")

# def get_id(screen_name):
#     try:
#         item = vk.users.get(user_ids=screen_name)
#         if 'deactivated' not in item:
#             return item[0]['id']
#     except Exception as e:
#         print(e.args)
# Нужно замапить screen_name с их vk_id, поэтому придется обрабатывать по одному

social = pd.read_msgpack("assets/social.msg")

In [7]:
label_encoder = LabelEncoder()
# labels_trunc = ["Искусство", "Политика", "Финансы", "Стратегическое управление", "Юриспруденция", "Исследования и разработки",
#           "Промышленность", "Образование", "Здравоохранение", "Сельское хозяйство", 
#           "Государственное управление", "Реклама и маркетинг", "Инновации и модернизация", "Безопасность", 
#           "Военное дело", "Корпоративное управление", "Социальная защита", "Строительство", "Предпринимательство",
#           "Инвестиции"]
labels_trunc = labels
int_labels = label_encoder.fit_transform(labels_trunc)

def f(x):
    b = []
    for a in x.split(","):
        t = a.strip()
        if t == "Частный бизнес" or t == "Социальное предпринимательство":
            b.append("Предпринимательство")
        elif t == "Дошкольное образование/детский отдых":
            b.append("Образование")
        elif t in {'Журналистика', 'Управление персоналом', 'Управление рисками'}:
            pass
        else:
            b.append(t)
    return b[:6]
social.Интересы = social.Интересы.apply(f)
def g(x):
    if not x:
        return np.in1d(int_labels, []).astype('int')
    return np.in1d(int_labels, label_encoder.transform(x)).astype('int')
social['y'] = social.Интересы.apply(g)

### Corpora from created texts

In [8]:
corpora_class = CorporaClass()

for filename in categories:
    with open(f"assets/corpora/{filename}.txt") as f:
        corpora_class.add_to_corpora(f, filename)

pickle.dump(corpora_class.corpora, open("assets/corpora.p", "wb"))
pickle.dump(corpora_class.labels, open("assets/labels.p", "wb"))

corpora = pickle.load(open("assets/corpora.p", "rb"))
labels = pickle.load(open("assets/labels.p", "rb"))

### Corpora from user-generated content

In [9]:
corpora_user_gen = CorporaClass()
corpora_user_gen.labels = pickle.load(open("assets/labels_user_gen.p", "rb"))
corpora_user_gen.corpora = pickle.load(open("assets/corpora_user_gen.p", "rb"))

# r = ResultClass()
# r.publics_dict = json.load(open("assets/publics_dict.json"))
# delimiter = 0
# for i, row in tqdm.tqdm(list(social.iterrows())[delimiter:], total=len(social) - delimiter):
#     user_vk, user_fb = row.vk, row.fb
#     if str(user_vk) != "nan":
#         try:
#             r.parse_vk(user_vk, parse=True)
#         except: pass
#     if str(user_fb) != "nan":
#         try:
#             r.parse_fb(user_fb)
#         except KeyError:
#             pass
#     corpora_user_gen.add_to_corpora(r.texts, i)
#     r.texts = []

# pickle.dump(corpora_user_gen.corpora, open("assets/corpora_user_gen.p", "wb"))
# pickle.dump(corpora_user_gen.labels, open("assets/labels_user_gen.p", "wb"))

In [9]:
# def filter_corpora(corpora, labels=corpora_user_gen.labels, DELIM=200, k=0.8):
#     indexes = np.array(list(range(len(corpora))))
#     all_indexes = set()
#     for col in norm_names.values():
#         t = social.Интересы.apply(lambda s: col in s)
#         col_labels = list(set(t[t == True].index).intersection(labels))[:50]
#         all_indexes = all_indexes.union(indexes[np.in1d(labels, col_labels)])
#     all_indexes = list(all_indexes)
#     new_corpora = list(np.array(corpora)[all_indexes])
#     new_labels = list(np.array(labels)[all_indexes])
#     new_corpora = [a[:DELIM] for a in new_corpora]
#     new_corpora, test_corpora, new_labels, test_labels = train_test_split(new_corpora, new_labels, train_size=k)
#     return new_corpora, test_corpora, new_labels, test_labels

# new_corpora, test_corpora, new_labels, test_labels = filter_corpora(corpora_user_gen.corpora, corpora_user_gen.labels)

# d = {}
# for col in norm_names.values():
#     t = social.Интересы.apply(lambda s: col in s[:3])
#     col_labels = list(set(t[t == True].index).intersection(new_labels))
#     d[col] = [label for label in col_labels if label not in list(itertools.chain(*(v for v in d.values())))][:10]
# d = {a: [int(c) for c in b] for (a, b) in d.items()}
# json.dump(d, open("assets/d.json", "w"))
    
# new_corpora = np.array(new_corpora)
# for col, labels in d.items():
#     with open(f"assets/corpora/{norm_names_reversed[col]}.txt", "a") as f:
#         for line in list(itertools.chain(*(item for item in new_corpora[np.in1d(new_labels, d[col])]))):
#             _ = f.write(line)

# Это те labels, которые мы приписали к corpora и должны учесть в y
d = json.load(open("assets/d.json"))

In [10]:
vectorizer = TfidfVectorizer(tokenizer=corpora_class.full_process, 
                             max_df=0.5, 
                             min_df=3, 
                             sublinear_tf=True, 
                             ngram_range=(1, 1))

# vectorizer = TfidfVectorizer(tokenizer=corpora_class.full_process, 
#                              max_df=0.55, 
#                              min_df=2, 
#                              sublinear_tf=True, 
#                              ngram_range=(1, 1))

docterm_matrix = vectorizer.fit_transform(list(itertools.chain(*(doc for doc in corpora))))


pickle.dump(docterm_matrix, open("assets/docterm_matrix.p", "wb"))
pickle.dump(vectorizer, open("assets/vectorizer.p", "wb"))

In [13]:
docterm_matrix_user_gen = vectorizer.transform(list(itertools.chain(*(doc for doc in new_corpora))))

In [11]:
docterm_matrix = pickle.load(open("assets/docterm_matrix.p", 'rb'))
vectorizer = pickle.load(open("assets/vectorizer.p", "rb"))

In [12]:
DELIM = 1300
NUM_OF_CLASSES = 22
vector_size = docterm_matrix[0].shape[1]

In [13]:
corpora_lengths = []
for i, x in enumerate(corpora):
    corpora_lengths.append(len(x))

#### y for created texts

In [14]:
y = []
for (i, item), cat in zip(zip(range(len(corpora)), corpora_lengths), categories):
    for _ in range(item):
        y_ = np.zeros(NUM_OF_CLASSES)
        t = np.zeros(NUM_OF_CLASSES)
        for arr in (social.loc[d[norm_names[cat]], 'y'] * 0.016):
            t = t + arr
        y_ = y_ + t
        y_[i] = 1
        y.append(y_)
y = np.array(y)

#### y for user-gen

In [52]:
y_user_gen = np.empty((docterm_matrix_user_gen.shape[0], NUM_OF_CLASSES))
y_ = np.array([np.array(a) for a in social.loc[new_labels, 'y']])
ind_arr = np.cumsum(np.array(corpora_lengths_user_gen))
int_a, int_b = 0, ind_arr[0]
for i in range(len(ind_arr) - 1):
    y_user_gen[int_a:int_b] = np.array([y_[i] for _ in range(int_b - int_a)])
    int_a, int_b = int_b, ind_arr[i + 1]

In [25]:
X = docterm_matrix.toarray()

In [15]:
X_train, X_test, y_train, y_test = train_test_split(docterm_matrix, y, test_size=0.2)

In [16]:
input_vec = Input(shape=(vector_size,))
l1 = Dense(128, activation='relu')(input_vec)
l2 = Dense(64, activation='relu')(l1)
l3 = Dropout(0.2)(l2)
l4 = Dense(96, activation='sigmoid')(l3)
output = Dense(NUM_OF_CLASSES, activation='sigmoid')(l4)

# input_vec = Input(shape=(vector_size,))
# l1 = Dense(128, activation='relu')(input_vec)
# l2 = Dense(64, activation='relu')(l1)
# l3 = Dropout(0.3)(l2)
# l4 = Dense(32, activation='relu')(l3)
# l5 = Dense(96, activation='sigmoid')(l4)
# output = Dense(NUM_OF_CLASSES, activation='sigmoid')(l5)

classifier = Model(input_vec, output)
classifier.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [17]:
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

# classifier.load_weights('vk_texts_classifier.h5')

# for using sparse vectors
def nn_batch_generator(X_data, y_data, batch_size):
    X_data, y_data, _, _ = train_test_split(X_data, y_data, train_size=1.0)
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].todense()
        y_batch = y_data[index_batch]
        counter += 1
        yield np.array(X_batch), y_batch
        if (counter > number_of_batches):
            counter=0

classifier.fit(X_train.toarray(), 
               y_train, 
               validation_data=(X_test.toarray(), y_test), 
               batch_size=196, 
               epochs=60, 
               class_weight='balanced',
               callbacks=[tbCallBack]
              )
# classifier.fit_generator(nn_batch_generator(X_train, y_train, 196),
#                validation_data=nn_batch_generator(X_test, y_test, 196),
#                steps_per_epoch=X_train.shape[0] // 196,
#                validation_steps=X_test.shape[0] // 196,
#                epochs=60, 
#                class_weight='balanced',
#                callbacks=[tbCallBack]
#               )

Train on 52744 samples, validate on 13186 samples
Epoch 1/60
52744/52744 [==============================] - 51s - loss: 4.6251 - categorical_accuracy: 0.0376 - val_loss: 4.6025 - val_categorical_accuracy: 0.0376
Epoch 2/60
52744/52744 [==============================] - 57s - loss: 4.5956 - categorical_accuracy: 0.0401 - val_loss: 4.6028 - val_categorical_accuracy: 0.0573
Epoch 3/60
52744/52744 [==============================] - 55s - loss: 4.5949 - categorical_accuracy: 0.0400 - val_loss: 4.6027 - val_categorical_accuracy: 0.0376
Epoch 4/60
52744/52744 [==============================] - 51s - loss: 4.5940 - categorical_accuracy: 0.0435 - val_loss: 4.5998 - val_categorical_accuracy: 0.0376
Epoch 5/60
52744/52744 [==============================] - 51s - loss: 4.5921 - categorical_accuracy: 0.0413 - val_loss: 4.5982 - val_categorical_accuracy: 0.0376
Epoch 6/60
52744/52744 [==============================] - 54s - loss: 4.5885 - categorical_accuracy: 0.0417 - val_loss: 4.5928 - val_categor

52744/52744 [==============================] - 53s - loss: 3.0124 - categorical_accuracy: 0.7619 - val_loss: 3.7622 - val_categorical_accuracy: 0.5095
Epoch 52/60
52744/52744 [==============================] - 54s - loss: 3.0012 - categorical_accuracy: 0.7661 - val_loss: 3.7620 - val_categorical_accuracy: 0.5136
Epoch 53/60
52744/52744 [==============================] - 45s - loss: 2.9899 - categorical_accuracy: 0.7696 - val_loss: 3.7563 - val_categorical_accuracy: 0.5158
Epoch 54/60
52744/52744 [==============================] - 45s - loss: 2.9784 - categorical_accuracy: 0.7726 - val_loss: 3.7628 - val_categorical_accuracy: 0.5133
Epoch 55/60
52744/52744 [==============================] - 45s - loss: 2.9667 - categorical_accuracy: 0.7773 - val_loss: 3.7616 - val_categorical_accuracy: 0.5150
Epoch 56/60
52744/52744 [==============================] - 50s - loss: 2.9567 - categorical_accuracy: 0.7799 - val_loss: 3.7658 - val_categorical_accuracy: 0.5200
Epoch 57/60
52744/52744 [=========

In [18]:
classifier.save("assets/vk_texts_classifier.h5")

In [19]:
classifier = load_model("assets/vk_texts_classifier.h5")

# Result

In [20]:
result_class = ResultClass()
result_class.parse_vk(134070307)
corpora_class = CorporaClass()
corpora_class.add_to_corpora(result_class.texts, '')

1-th public have been parsed. (58219172)
2-th public have been parsed. (28905875)
3-th public have been parsed. (139246969)
4-th public have been parsed. (55284725)
5-th public have been parsed. (84926122)


In [21]:
transformed = result_class.vectorizer.transform(corpora_class.corpora[0])
verdict1 = normalize(np.sum(result_class.classifier.predict(transformed.toarray()), axis=0).reshape(1, -1))[0]
verdict2 = normalize(result_class.classifier.predict(np.sum(transformed, axis=0)).reshape(1, -1))[0]

In [29]:
t = " ".join(corpora_class.corpora[0])
normalize(result_class.classifier.predict(np.sum(result_class.vectorizer.transform([t]), axis=0)).reshape(1, -1))[0]

array([  1.58050482e-03,   6.46794308e-03,   7.65688455e-05,
         1.75090730e-01,   1.35872543e-01,   8.20550978e-01,
         5.53579675e-03,   4.65691447e-01,   8.31513375e-07,
         3.94662988e-04,   1.56653114e-05,   7.64702559e-02,
         4.29760478e-02,   1.14148781e-01,   3.48257017e-03,
         8.69610987e-04,   1.33026719e-01,   6.09717565e-03,
         5.01470780e-03,   1.34844348e-01,   3.38851464e-06,
         6.28931075e-02], dtype=float32)

In [32]:
%%timeit
normalize(result_class.classifier.predict(np.sum(result_class.vectorizer.transform([" ".join(corpora_class.corpora[0])]), axis=0)).reshape(1, -1))[0]

2.13 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
transformed.shape

(1518, 19770)

In [34]:
%%timeit
normalize(np.sum(result_class.classifier.predict(transformed.toarray()), axis=0).reshape(1, -1))[0]

30.3 ms ± 4.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
norm_categories = np.array(list(norm_names.values()))

In [39]:
dict_for_mean = []

labels_ = np.array(test_labels)
corpora_ = np.array(test_corpora)

acc_d = {}
for col in norm_names.values():
    t = social.Интересы.apply(lambda s: col in s)
    col_labels = np.array(list(set(t[t == True].index).intersection(labels_)))
    c_0 = 0
    c_1 = 0
    c_2 = 0
    for item in corpora_[np.in1d(labels_, col_labels)]:
        t = np.sum(classifier.predict(vectorizer.transform(item).toarray()), axis=0)
        dict_for_mean.append(t)
        pred_categories = norm_categories[t.argsort()[::-1][:8]]
        if col in pred_categories[:2]:
            c_0 += 1
        if col in pred_categories[:5]:
            c_1 += 1
        if col in pred_categories:
            c_2 += 1
    l = len(col_labels)
    if l == 0:
        l = 1
    col, l, c_0 / l, c_1 / l, c_2 / l
    acc_d[col] = c_2 / l
    
    
#     sorted(np.sum(classifier.predict(vectorizer.transform(corpora_user_gen.corpora[5]).toarray()), axis=0), reverse=True)[:3]

('Искусство', 9, 0.2222222222222222, 0.2222222222222222, 0.3333333333333333)

('Политика', 17, 0.23529411764705882, 0.23529411764705882, 0.29411764705882354)

('Финансы', 6, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333)

('Стратегическое управление',
 27,
 0.25925925925925924,
 0.6666666666666666,
 0.8888888888888888)

('Юриспруденция',
 6,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333)

...

('Социальная защита',
 3,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333)

('Строительство', 8, 0.125, 0.125, 0.125)

('Предпринимательство', 36, 0.25, 0.6666666666666666, 0.9722222222222222)

('Спорт', 1, 0.0, 0.0, 0.0)

('Инвестиции',
 21,
 0.19047619047619047,
 0.19047619047619047,
 0.3333333333333333)

In [40]:
np.mean(list(acc_d.values()))

0.45481123580974059

In [41]:
list(zip(categories, [np.mean(a) for a in np.array([np.array(b) for b in dict_for_mean]).T]))

[('art', 0.11978178),
 ('politics', 0.2571713),
 ('finances', 0.15092145),
 ('strateg_management', 0.27437153),
 ('law', 0.14367713),
 ('elaboration', 0.2649323),
 ('industry', 0.14514737),
 ('education', 0.29252267),
 ('charity', 0.11408015),
 ('public_health', 0.16153345),
 ('agriculture', 0.21364743),
 ('government_management', 0.26083517),
 ('smm', 0.23419189),
 ('innovations', 0.23632175),
 ('safety', 0.15871054),
 ('military', 0.059539381),
 ('corporative_management', 0.25076938),
 ('social_safety', 0.12933674),
 ('building', 0.14101389),
 ('entrepreneurship', 0.26689848),
 ('sport', 0.21294023),
 ('investitions', 0.18971834)]

In [23]:
# social = pd.read_excel("social.xlsx", names=['id', 'name', 'fb', 'ok', 'vk', 'email', 'twitter'])
# social.set_index('id', inplace=True)
# social = social[social.vk.notnull()]
# social.vk = social.vk.apply(lambda x: str(x).split("/")[-1])
# df = social[social.vk.notnull()]
# df = df[['name', 'vk']]

# def get_id(screen_name):
#     try:
#         item = vk.users.get(user_ids=screen_name)
#         if 'deactivated' not in item:
#             return item[0]['id']
#     except Exception as e:
#         print(e.args)
# # Нужно замапить screen_name с их vk_id, поэтому придется обрабатывать по одному

# df['vk_id'] = df.vk.progress_apply(get_id)
# df.dropna(inplace=True)
# df.vk_id = df.vk_id.astype('int')
# df.to_msgpack("df.msg")
df = pd.read_msgpack("assets/df.msg")
known = pd.read_csv("assets/known_users.csv")
known = known.merge(df[['vk_id']], left_on='Leader-ID', right_index=True, how='left')

In [83]:
from line_profiler import LineProfiler

In [88]:
lp = LineProfiler()

In [89]:
lp.add_function(corpora_class.full_process)

In [91]:
lp.run("corpora_class.full_process(corpora[0][0])")

In [93]:
lp.print_stats()

Timer unit: 1e-06 s

Total time: 0.017674 s
File: /Users/george/Dropbox/Projects/vk_text_classifier/util.py
Function: full_process at line 36

Line #      Hits         Time  Per Hit   % Time  Line Contents
    36                                               @staticmethod
    37                                               def full_process(text, tokenizer=tokenizer, morph=morph, ru_pattern=ru_pattern):
    38                                                   # Clear text from punctuation etc.'''
    39         1          110    110.0      0.6          tokens = tokenizer.tokenize(text)
    40                                           
    41                                                   # Turn tokens into normal form excluding non-nouns or verbs
    42         1            1      1.0      0.0          processed = []
    43       102           77      0.8      0.4          for token in tokens:
    44       101         8888     88.0     50.3              morphed = morph.parse(token)[

In [57]:
import requests

In [103]:
%%timeit
requests.post("http://78.155.197.212:9999/get_result", json={"name": "DimaB", "user_vk": int(np.random.choice(t)), "user_fb": "blag54"}).json()

18.1 s ± 3.33 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [101]:
import vk_api
vk_s = vk_api.VkApi()
vk = vk_s.get_api()
t = vk.friends.get(user_id=118341119)['items']

In [56]:
transformed

<1518x19770 sparse matrix of type '<class 'numpy.float64'>'
	with 13149 stored elements in Compressed Sparse Row format>

In [37]:
offset = 0
for index, row in tqdm.tqdm(known.iloc[offset:, :].iterrows(), total=len(known) - offset):
    user_vk = row['vk_id']
    if str(user_vk) == "nan":
        user_vk = None
    user_fb = row['FB']
    if str(user_fb) == "nan":
        user_fb = None
    try:
        verdict = result_class.get_result(user_vk, user_fb)
        result_class.texts = []
        for cat, value in verdict:
            known.loc[index, cat] = value
    except ValueError:
        for cat in categories:
            known.loc[index, cat] = 0
    except IndexError:
        for cat in categories:
            known.loc[index, cat] = 0

  0%|          | 0/44 [00:00<?, ?it/s]

FB Parsing alya.blesk
FB Parse completed.
Added to corpora
Transformed corpora.


  2%|▏         | 1/44 [00:01<00:52,  1.21s/it]

FB Parsing zonovaangelina
FB Parse completed.
Added to corpora


  5%|▍         | 2/44 [00:01<00:38,  1.09it/s]

...

 95%|█████████▌| 42/44 [00:40<00:02,  1.32s/it]

FB Parse completed.
Added to corpora
Transformed corpora.
FB Parsing 100007131103601
FB Parse completed.


 98%|█████████▊| 43/44 [00:42<00:01,  1.26s/it]

Added to corpora
Transformed corpora.
VK Parsing 2416234.0
1-th public have been parsed. (31920990)
2-th public have been parsed. (1967021)
3-th public have been parsed. (227)
4-th public have been parsed. (8179697)
VK Parse completed.
FB Parsing dmitry.zemtsov
FB Parse completed.
Added to corpora
Transformed corpora.


100%|██████████| 44/44 [00:45<00:00,  1.98s/it]


In [38]:
norm_dict = {"values": []}
for name in known.ФИ:
    results = []
    for col in known[known.ФИ == name].iloc[:, 4:].columns:
        results.append({"name": norm_names[col], "value": float(known.loc[known.ФИ == name, col].values[0])})
    norm_dict['values'].append({"name": name, "results": results})

In [42]:
[(norm_categories[x.argsort()[-1]], norm_categories[x.argsort()[-2]]) for x in known.loc[:, categories].values]

[('Образование', 'Предпринимательство'),
 ('Строительство', 'Образование'),
 ('Инвестиции', 'Спорт'),
 ('Спорт', 'Предпринимательство'),
 ('Реклама и маркетинг', 'Образование'),
 ('Образование', 'Здравоохранение'),
 ('Государственное управление', 'Корпоративное управление'),
 ('Образование', 'Предпринимательство'),
 ('Образование', 'Промышленность'),
 ('Образование', 'Реклама и маркетинг'),
 ('Искусство', 'Исследования и разработки'),
 ('Спорт', 'Образование'),
 ('Образование', 'Предпринимательство'),
 ('Спорт', 'Предпринимательство'),
 ('Образование', 'Предпринимательство'),
 ('Инвестиции', 'Спорт'),
 ('Образование', 'Предпринимательство'),
 ('Образование', 'Предпринимательство'),
 ('Социальная защита', 'Образование'),
 ('Реклама и маркетинг', 'Образование'),
 ('Здравоохранение', 'Благотворительность'),
 ('Предпринимательство', 'Образование'),
 ('Предпринимательство', 'Образование'),
 ('Образование', 'Строительство'),
 ('Строительство', 'Исследования и разработки'),
 ('Здравоохранение

In [43]:
means = []
for col in known.iloc[:, 4:].columns:
    norm_names[col], known.loc[:, col].median()
    means.append(known.loc[:, col].median())

('Искусство', 0.11816754192113876)

('Политика', 0.1563805639743805)

('Финансы', 0.13835492730140686)

('Стратегическое управление', 0.2304811328649521)

('Юриспруденция', 0.07182104140520096)

...

('Социальная защита', 0.1679375171661377)

('Строительство', 0.19163548946380615)

('Предпринимательство', 0.3297472596168518)

('Спорт', 0.022987650707364082)

('Инвестиции', 0.20343999564647675)

In [53]:
# t is from user-gen up
means_2 = [(a + b) / 2 for a, b in zip([z[1] for z in t], [b for b in means])]

In [54]:
json.dump(dict(zip(categories, means_2)), open("margins.json", "w"))

In [27]:
known.to_csv("assets/known.csv")
json.dump(norm_dict, open("assets/temporary_result.json", "w"))

In [51]:
tt = []
for i, item in known.iterrows():
    print(item['ФИ'], end=" ")
    accepted_cols = []
    for col, margin in dict(zip(categories, means_2)).items():
        if item[col] > 1.1 * margin:
            accepted_cols.append(col)
    np.array(accepted_cols)[item[accepted_cols].argsort()[::-1]][:5]

Александрина Клюс 

array(['education', 'smm', 'entrepreneurship', 'strateg_management',
       'politics'],
      dtype='<U18')

Ангелина Зонова 

array(['education', 'entrepreneurship', 'smm', 'corporative_management',
       'building'],
      dtype='<U22')

Владислав Широков 

...

array(['smm', 'education', 'strateg_management', 'entrepreneurship',
       'corporative_management'],
      dtype='<U22')

Гнитько Ксения 

array(['charity', 'education', 'law', 'smm', 'industry'],
      dtype='<U9')

Дмитрий Земцов 

array(['strateg_management', 'entrepreneurship', 'elaboration', 'smm',
       'innovations'],
      dtype='<U18')

In [ ]:
import vk_api
from config import VK_TOKEN

In [ ]:
vk = vk_api.VkApi(token=VK_TOKEN)
vk = vk.get_api()

In [ ]:
t = [a['text'] for a in vk.newsfeed.search(q="корпоративное управление", count=200)['items']]

In [ ]:
with open('t.txt', 'w') as f:
    for line in t:
        _ = f.write(f'{line}\n')